In [946]:
import random
from random import randint
import pandas as pd

In [947]:
measurement_height = ["inches", "cm", "meters", "feet"]
items_for_rent_list = ["cars","books","computers","buses","houses","bikes"]
days_in_a_week = ["Sunday","Monday","Wednesday","Thursday","Friday","Saturday"]
measurement_liquid = ["ml", "millileters","litres", "L", "Lit","ounces","oz","gallons"]
containers_list = ["bottles","jars","tanks","tumblers","mugs","bowls"]  
measurement_weight = ["kgs","kilograms","kilos","pounds","mg","milligrams","grams","gm"]
vegetables = ["tomatoes","onions","capsicum","mushrooms","carrots"]
store_type = ["grocery store", "super market","salon", "shop","field","school"]
duration_list = ["days","weeks","years","months"]
activity_list = ["walk","run","jog","race","speed walk","brisk walk"]
measurement_distance = ["km","mts","cm","mm","miles"]
options_set = ["Both A and B","Both a and b","Both a and c","Both C and A", "Both Option 1 and Option 2","First letter of apple and first letter of banana","Both B and C","Options C and B","First letter of charlie and third letter of subway","All the above","None of the above"]
month_list = ["January","February","March","April","May","June","July","August","September","October","November","December"]

In [948]:
json_dataset = []
duplicity = 200

In [949]:
Options_set_0 = ["None of the above","Given options do not satisy","No valid option","None","Not a or b or c"] 
Options_set_1 = ["Options a and b","First letter of charlie and third letter of subway","First letter of apple and first letter of banana","Options a, b and c","All the above","Options b and c","Option d","Options a and c","None of the above","Given options do not satisy","No valid option"]
Options_set_2_ab = ["Both a and b","Both b and a"] 
Options_set_2_bc = ["Both b and c","Both c and b"]
Options_set_2_ac = ["Both a and c","Both c and a"]
Options_set_3 = ["All the above","Options a, b and c"]

In [950]:
def replace_parameters(template,parameters):
    for i in parameters:
        k = template.find("$",0,len(template))
        if k!=-1:
            template = template[:k]+i+template[k+2:]
    return template

In [951]:
#******************************************************************************************************#
#  Template 1 : Based on the Capacity of different vehicles.  Creating different set of options       *#
#******************************************************************************************************#
data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Vehicle': str},sheet_name='template_1')  
df = pd.DataFrame(data,columns=['Vehicle'])   # Ordered vehicles by capacity in increasing order

template_1 = "If a group of people need $1 $B each having maximum capacity of $2 to travel, then which of the following options could be the number of $C having maximum capacity of $3 would they need for the travel?"

Vehicle_list = df['Vehicle'].tolist()
i=0
while i<duplicity:
    QA = {}
    
    # parameters
    param_1 = randint(2,100)
    B = random.choice(Vehicle_list)
    param_2 = randint(4,100)
    C_list = []
    C_list = Vehicle_list.copy()
    C_list.remove(B)
    C = random.choice(C_list)
    index_1 = Vehicle_list.index(B)
    index_2 = Vehicle_list.index(C)
    if index_1 < index_2 : #Second vehicle should ideally have more capacity than first
        param_3 = randint(param_2+1,1000)
    else:                  #Second vehicle capacity should be less than first
        param_3 = randint(2,param_2-1)
        
    parameters = [str(param_1),B,str(param_2),C,str(param_3)]
  
    #questions
    QA["question"] = replace_parameters(template_1,parameters)    
    
    # answers 
    total_no_of_people = param_1*param_2
    # maximum number of vehicles of type 2 required = total no of people (assuming each person can take one vehicle)
    max_1 = total_no_of_people
    #calculate the minimum number of vehicles of type 2 required
    if total_no_of_people % param_3 == 0:
        min_1 = int(total_no_of_people / param_3) 
        option_invalid = randint(0,min_1-1)
    else:
        min_1 = int(total_no_of_people / param_3)+1
        option_invalid = randint(0,min_1)  

    
    # Generate Answers and Options for any of the following sets
    #------------------------------------------------------------
    # To randomly choose options set for each question
    # 0 - Options with no correct answer
    # 1 - Options with one correct answer 
    # 2 - Options with two correct answers
    # 3 - Options with three correct answers
    
    if max_1-min_1>3: #checking if atleast three answers are possible
        ans_1 = randint(min_1+1,max_1) 
        ans_2 = ans_1
        while ans_1 == ans_2:
            ans_2 = randint(min_1+1,max_1)
        ans_3 = ans_2
        while ans_3 == ans_1 or ans_3 == ans_2:
            ans_3 = randint(min_1+1,max_1)
    else:
        continue
        i = i-1
    
    
    # generate options set with incorrect answers 
    options_set_no_ans = [max_1+randint(2,1000), max_1+1,0, int(-1 *randint(2,1000)),option_invalid] 
    if param_2 != param_3:
        options_set_no_ans.append(param_1)
    if param_2 > param_3:  #if capacity of vehicle 1 is > capacity of vehicle 2
        options_set_no_ans.append(randint(1,param_1-1))  #answer cannot be less than param_1
    
    
    choose_options_set = randint(0,3)  
    if choose_options_set == 0: # no correct answers
        set_1 = []
        set_1 = options_set_no_ans.copy() 
        option_1 = random.choice(set_1)
        set_1.remove(option_1)
        option_2 = random.choice(set_1)
        set_1.remove(option_2)
        option_3 = random.choice(set_1)
        options_list = [option_1,option_2,option_3]
        option_4 = random.choice(Options_set_0)
        options_list.append(option_4)
        ans_t1 = option_4
        
    elif choose_options_set == 1:  # one correct answer 
        options_list = [ans_1,option_invalid,random.choice(options_set_no_ans)]    
        random.shuffle(options_list)
        options_list.append(random.choice(Options_set_1))
        ans_t1 = ans_1
        
    elif choose_options_set == 2: # two correct answers   
        options_list = [ans_1,ans_2,random.choice(options_set_no_ans)]
        random.shuffle(options_list)
        if (options_list[0] == ans_1 or options_list[0] == ans_2) and (options_list[1] == ans_1 or options_list[1] == ans_2):
            option_4 = random.choice(Options_set_2_ab)
        elif (options_list[1] == ans_1 or options_list[1] == ans_2) and (options_list[2] == ans_1 or options_list[2] == ans_2):
            option_4 = random.choice(Options_set_2_bc)         
        else:
            option_4 = random.choice(Options_set_2_ac)
        options_list.append(option_4)
        ans_t1 = option_4
    else:            # all correct answers
        options_list = [ans_1,ans_2,ans_3]
        random.shuffle(options_list)
        option_4 = random.choice(Options_set_3)
        options_list.append(option_4)
        ans_t1 = option_4        
                 
    QA["options"] = options_list
    QA["answer"] = ans_t1
    

    
    json_dataset.append(QA)
    
    i=i+1;


In [952]:
#***********************************************************************************************************#
#  Template 2 : Based on  rankings (1st, 2nd, 3rd etc...) and comparision keywords (more than, less than)  *#
#      For example: 2nd shortest person's height is less than 3rd shortest person's height                 *#                         
#***********************************************************************************************************#

data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Names': str},sheet_name='names')  
df = pd.DataFrame(data,columns=['Names'])   
names = df['Names'].tolist()  

keywords_t2 = ["less","more"]
template_2 = "In a list of $1 shortest people in the world, if $A's height is $B than $x times of the $a$q shortest person whose height is $2 inches and $C than $y times of $b$r shortest person whose height is $3 inches. Which of the following can be the height of $A?"

def find_2(x1):
    if x1==1:
        res="st"
    elif x1==2:
        res = "nd"
    elif x1==3:
        res = "rd"
    else:
        res="th"
    return res      
i=0
while i<duplicity:
    QA = {}
    
    # parameters
    param_1 = randint(10,100)
    A = random.choice(names)
    B = random.choice(keywords_t2)
    if B=="more":
        C = "less"
        x = randint(2,20) # y>x , so param_3 > param_2
        y = randint(x+1,40)
        a = randint(1,param_1-1)
        b = randint(a+1,param_1)
        param_2 = randint(5,20)
        param_3 = randint(param_2+1,22)   
        
    else:
        C="more"
        y = randint(2,20) # x> y, so param_2 > param_3
        x = randint(y+1,40)
        b = randint(1,param_1-1)
        a = randint(b+1,param_1) 
        param_3 = randint(5,20)
        param_2 = randint(param_3+1,22) 
    
    q = find_2(a)
    r = find_2(b)
    parameters = [str(param_1),str(A),str(B),str(x),str(a),str(q),str(param_2),str(C),str(y),str(b),str(r),str(param_3),str(A)]
    
    #questions
    QA["question"] = replace_parameters(template_2,parameters)    

    # answers
    if B=="more":
        ans_t2 = randint(param_2*x+1,param_3*y-1)
        other_options = [param_3, param_2,param_2*x,param_3*y,0]
    else:
        ans_t2 = randint(param_3*y+1,param_2*x-1)
        other_options = [param_1, param_2,param_2*x,param_3*y,0]
    
    #options
    Option_2 = random.choice(other_options)
    Option_3 = random.choice(other_options)
    options_list = [ans_t2,Option_2,Option_3]
    random.shuffle(options_list)
    option_4 = random.choice(options_set)
    options_list.append(option_4)
    QA["options"] = options_list
    
    QA["answer"] = ans_t2
    

    json_dataset.append(QA)

    i+=1
    


In [953]:
#***********************************************************************************************************#
#  Template 3 : Based on the occurence of events                                                           *#
#***********************************************************************************************************#

data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Event':str,'Type':str,'Category':str},sheet_name='template_3')  
df = pd.DataFrame(data,columns=['Event','Type','Category'])


template_3 = "If $A happens in the $y $1. When could be the $y in which $B took place?"

def find(lst,a):
    result = []
    j=0;
    for x in lst:
        if x == a:
            result.append(j)
        j = j+1
    return result
i=0
list_A = df['Event'].tolist()
list_B = df['Type'].tolist()
list_C = df['Category'].tolist()
while i<duplicity:
    QA = {}
    
    # parameters
    index_1= randint(0,len(list_A)-1)
    event_A = list_A[index_1]
    y = list_C[index_1]
    if y=="year":
        param_1 = randint(802,2018)
    elif y=="month":
        param_1 = randint(3,10)
    else:
        param_1 = randint(3,22)
    list_X =[] 
    list_X = find(list_B,list_B[index_1])
    list_X.remove(index_1)
    index_event_B = random.choice(list_X)
    event_B = list_A[index_event_B]

    parameters = [str(event_A),y,str(param_1),y,str(event_B)]
    
    #questions
    QA["question"] = replace_parameters(template_3,parameters)    
   
    # answer
    if index_1>index_event_B: #event B happened before event A
        if y=="year":
            ans_t3 = randint(800,param_1-1)
            option_2 = randint(param_1+1,2020)
        elif y=="month":
            ans_t3 = randint(1,param_1-1)
            option_2 = randint(param_1+1,12)
        else:
            ans_t3 = randint(1,param_1-1)
            option_2 = randint(param_1+1,24)
        
       
    else: #event B happened after event A
        if y=="year":
            ans_t3 = randint(param_1+1,2020)
            option_2 = randint(1,param_1-1)
        elif y=="month":
            ans_t3 = randint(param_1+1,12)
            option_2 = randint(1,param_1-1)
        else:
            ans_t3 = randint(param_1+1,24)
            option_2 = randint(1,param_1-1)
           
    #options
    options_list = [ans_t3, option_2, param_1, 2020 + randint(1,1000)]
    random.shuffle(options_list)
    QA["options"] = options_list 
    QA["answer"] = ans_t3

    
    json_dataset.append(QA)
    
    i+=1;

In [954]:
#***********************************************************************************************************#
#  Template 4 : Based on the keywords (above, before, ahead etc) mentioned in Keyword_4 list               *#                      *#
#***********************************************************************************************************#
data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Names': str},sheet_name='names')  
df = pd.DataFrame(data,columns=['Names'])
type_of_competition = ["race","competition","sport"]
keyword_4 = ["ahead of", "below", "before", "after", "beyond", "above", "later than","behind"]

template_4 = "In a $X, if $A scores $1 points and is $Z $B. Which of the following can be the score of $B?"

i=0
while i<duplicity:
    QA = {}
    
    # parameters
    competition = random.choice(type_of_competition)
    list_A = df['Names'].tolist()
    obj_A = random.choice(list_A)
    list_B =[] 
    list_B = list_A.copy() 
    list_B.remove(obj_A)
    obj_B = random.choice(list_B)
    param_1 = randint(1,100000)
    keyword = random.choice(keyword_4)

    parameters = [competition,obj_A,str(param_1),keyword,obj_B,obj_B]
    
    #questions
    QA["question"] = replace_parameters(template_4,parameters)    
   
    # answer
    if keyword == "ahead of" or keyword == "above" or keyword == "before" or keyword == "beyond": 
        ans_t4 = randint(0,param_1-1)
        option_2 = randint(param_1+1,1000000)
        option_3 = -1 * randint(param_1+1,1000000)
    else:
        ans_t4 = randint(param_1+1,1000000)
        option_2 = randint(0,param_1-1)
        option_3 = -1 * randint(0,param_1-1)
    
    #options
    options_list = [ans_t4, option_2, option_3]
    option_4 = random.choice(options_set)
    random.shuffle(options_list)
    options_list.append(option_4)
    QA["options"] = options_list 
    QA["answer"] = ans_t4

    
    json_dataset.append(QA)
    
    i+=1;

In [955]:
#***********************************************************************************************************#
#  Template 5 : Based on the the price of different items                                                  *#
#***********************************************************************************************************#
data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Price':str},sheet_name='template_5')  
df = pd.DataFrame(data,columns=['Price']) # Sorted as per descendig order of price

template_5 = "Cost of $A is $1 dollars. How much could be the cost of $B?"

i=0
list_A = df['Price'].tolist()
while i<duplicity:
    QA = {}
            
    # parameters
    obj_A = random.choice(list_A)
    list_B= []
    list_B = list_A.copy()
    list_B.remove(obj_A)    
    obj_B = random.choice(list_B)
    param_1 = randint(3,100000)
        
    parameters = [obj_A,str(param_1),obj_B]
    
    #questions
    QA["question"] = replace_parameters(template_5,parameters)    

    # answer
    index_1 = list_A.index(obj_A)
    index_2 = list_A.index(obj_B)
    if index_1 < index_2:   # Object A is expensive than Object B
        ans_t5 = randint(1,param_1-1)
        option_2 = randint(param_1+1,1000000)
        option_3 = -1 * randint(param_1+1,1000000)
    else:               # Object B is expensive than Object A 
        ans_t5 = randint(param_1+1,1000000)
        option_2 = randint(0,param_1-1)
        option_3 = -1 * randint(0,param_1-1)
    
    #options
    options_list = [ans_t5, option_2, option_3]
    option_4 = random.choice(options_set)
    random.shuffle(options_list)
    options_list.append(option_4)
    QA["options"] = options_list 
    QA["answer"] = ans_t5

    json_dataset.append(QA)
    
    i+=1;

In [956]:
#***********************************************************************************************************#
#  Template 6 : Based on the speed of different modes of transport                                         *#
#***********************************************************************************************************#
data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Place':str,'Vehicle':str},sheet_name='template_6')  
df = pd.DataFrame(data,columns=['Place','Vehicle']) #Vehicles sorted in the ascending order of speed
data_1 = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Names':str},sheet_name='names')  
df_1 = pd.DataFrame(data_1,columns=['Names'])

time_list = ["hours","days","years","seconds","minutes","months"]
template_6 = "If $A takes $1 $B to reach $C by $X, then how long in $B it would take to travel by $Y?"

i=0
A_list = df_1['Names'].tolist()
C_list = df['Place'].tolist()
X_list = df['Vehicle'].tolist()

while i<duplicity:
    QA = {}
            
    # parameters 
    A = random.choice(A_list)
    param_1 = randint(4,10000)
    B = random.choice(time_list)    
    C = random.choice(C_list)
    X = random.choice(X_list)
    Y_list = []
    Y_list = X_list.copy()
    Y_list.remove(X)
    Y = random.choice(Y_list)
        
    parameters = [A,str(param_1),B,C,X,B,Y]
    
    #questions
    QA["question"] = replace_parameters(template_6,parameters)    

    # answer
    index_1 = X_list.index(X)
    index_2 = X_list.index(Y)
    if index_1>index_2: #X is faster than Y
        ans_t6 = randint(0,param_1-1)
        option_2 = randint(param_1+2,100001)
        options_list_no_ans = [0,param_1+1, -1 *option_2]
        option_3 = random.choice(options_list_no_ans )
    else: #Y is faster than X
        ans_t6 = randint(param_1+1,100001)
        option_2 = randint(0,param_1-2)
        options_list_no_ans = [0,param_1-1, -1 *option_2]
        option_3 = random.choice(options_list_no_ans )
    
    #options
    options_list = [ans_t6, option_2, option_3]
    random.shuffle(options_list)
    option_4 = random.choice(options_set)
    options_list.append(option_4)
    QA["options"] = options_list 
    QA["answer"] = ans_t6
  
    json_dataset.append(QA)
    
    i+=1;

In [957]:
#***********************************************************************************************************#
#  Template 7 : Based on keywords mentioned in keywords_7_2  (leave, join , enter, exit etc..)              #
#***********************************************************************************************************#
data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Place':str},sheet_name='template_7')  
df = pd.DataFrame(data,columns=['Place'])
data_1 = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Category':str},sheet_name='human_category')  
df_1 = pd.DataFrame(data_1,columns=['Category'])

clock = ["am","pm"]
keywords_7_1 = ["Some","Few","A small number of","Less number of"]
keywords_7_2 = ["left","leave","departed","are withdrawn from","are sent out","exit","joined","entered","are merged"] 
split_index = 5 # in the above list elements in indexes 0 to split corresponds to subtraction, elements > split index corresponds to addition 
template_7 = "There are $1 $A in the $B at $t $x. $k $A $l the $B at $s $y. What could be the number of $A in the $B after $s $y?"

i=0
A_list = df_1['Category'].tolist()
B_list =df['Place'].tolist()
while i<duplicity:
    QA = {}
            
    # parameters
    param_1 = randint(4,9999)
    A = random.choice(A_list)
    B = random.choice(B_list)
    x = random.choice(clock)
    y = random.choice(clock)
    if (x=="am" and y=="am") or (x=="pm" and y=="pm"):
        t = randint(1,10)
        s = randint(t,11)
    else:
        t = randint(1,12)
        s = randint(1,12)
    k = random.choice(keywords_7_1)
    l = random.choice(keywords_7_2)
        
    parameters = [str(param_1),A,B,str(t),x,k,A,l,B,str(s),y,A,B,str(s),y]
    
    #questions
    QA["question"] = replace_parameters(template_7,parameters)    

    # answer
    index_1 = keywords_7_2.index(l)
    if index_1 > split_index:    #  Add the no_of people entering
        ans_t7 = randint(param_1+1,10000)
        option_2 = randint(0,param_1-1)
    else:                       # Subtract the no_of people leaving
        ans_t7 = randint(0,param_1-1)
        option_2 = randint(param_1+1,10000)
    
    options_no_ans = [-1*option_2,0,-1*randint(1,10)*ans_t7]
    option_3 = random.choice(options_no_ans)
    
    #options
    options_list = [ans_t7, option_2,option_3,param_1]
    random.shuffle(options_list)
    QA["options"] = options_list 
    QA["answer"] = ans_t7

    json_dataset.append(QA)
    
    i+=1;

In [958]:
#***********************************************************************************************************#
#  Template 8 : Based on the keywords less than, at most, at least etc..                                   *#
#***********************************************************************************************************#
data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Item': str,'Container':str},sheet_name='template_8')  
df = pd.DataFrame(data,columns=['Item','Container'])

keywords_8= ["a maximum of", "less than", "at most", "greater than","at least", "a minimum of","more than"]
split_index = 2
template_8= "There are $1 $A in $B. If $k $2 $A are lost, then how many $A will be remaining in the $B? "

A_list = df['Item'].tolist()
B_list =df['Container'].tolist()
i=0
while i<duplicity:
    QA = {}
            
    # parameters
    index = randint(0,len(A_list)-1)
    param_1 = randint(5,9999)
    A = A_list[index]
    B = B_list[index]
    k = random.choice(keywords_16)
    param_2 = randint(2,param_1-2) 
        
    parameters = [str(param_1),A,B,k,str(param_2),A,A,B]
    
    #questions
    QA["question"] = replace_parameters(template_8,parameters)    

    # answer
    index_1 = keywords_8.index(k)
    if k == "more than":    #remaining items <  initial no of items - items lost  
        ans_t8 = randint(0,param_1-param_2-1)
        option_3 = param_1-param_2
    elif k == "less than":  #remaining items >  initial no of items - items lost 
        ans_t8 = randint(param_1-param_2+1,param_1)
        option_3 = param_1-param_2
    elif index_1 > split_index:   #remaining items <=  initial no of items - items lost  
        ans_t8 = randint(0,param_1-param_2)
        option_3 = randint(param_1-param_2+1,param_1)
    else:                 #remaining items >=  initial no of items - items lost       
        ans_t8 = randint(param_1-param_2,param_1)
        option_3 = randint(0,param_1-param_2-1)

    #options
    options_list = [ans_t8,param_1,option_3]
    random.shuffle(options_list)
    option_4 = random.choice(options_set)
    options_list.append(option_4)
    QA["options"] = options_list 
    QA["answer"] = ans_t8

    json_dataset.append(QA)
    
    i+=1;

In [959]:
#***********************************************************************************************************#
#  Template 9 :  Larger vs Smaller objects                                                                 *#
#***********************************************************************************************************#
data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Item':str,'Use':str},sheet_name='template_9')  
df = pd.DataFrame(data,columns=['Item','Use'])
data_1 = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Names':str},sheet_name='names')  
df_1 = pd.DataFrame(data_1,columns=['Names'])

template_9 = "If $A needs $1 $B to $C, then how many $D will be needed to $C?"

i=0
def find(lst,a):
    result = []
    j=0;
    for x in lst:
        if x == a:
            result.append(j)
        j = j+1
    return result

names =df_1['Names'].tolist()
B_list =df['Item'].tolist() #items
C_list = df['Use'].tolist() # use case of item B
while i<duplicity:
    QA = {}           
    # parameters
    A = random.choice(names)
    param_1 = randint(2,10000)
    index = randint(0,len(B_list)-1)
    B = B_list[index]
    C = C_list[index] 
    D_list = []
    D_list = find(C_list,C)    # list of item indexes with same use case
    D_list.remove(index)
    index_choice = random.choice(D_list)
    D = B_list[index_choice]
        
    parameters = [A,str(param_1),B,C,D,C]
    
    #questions
    QA["question"] = replace_parameters(template_9,parameters)    

    # answer
    index_1 = index
    index_2 = index_choice
    if index_2>index_1: # Second item (D) is bigger than first item(B), so need less number of D items than B
        ans_t9 = randint(1,param_1-1)
        option_3 = randint(param_1+1,100000)
    else:
        ans_t9 = randint(param_1+1,100000)
        option_3 = randint(1,param_1-1)

    #options
    options_list = [ans_t9,param_1,option_3]
    random.shuffle(options_list)
    option_4 = random.choice(options_set)
    options_list.append(option_4)
    QA["options"] = options_list 
    QA["answer"] = ans_t9
  
    json_dataset.append(QA)
    
    i+=1;

In [960]:
#***********************************************************************************************************#
#  Template 10 : Unequal division (Change in Length)                                                       *#
#***********************************************************************************************************#
data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Item':str},sheet_name='template_10')  
df = pd.DataFrame(data,columns=['Item'])



template_10 = "A $A is divided into $1 parts. Length of one of the parts is $2 $k. What could be the length of $A before division in $k? "

i=0
A_list =df['Item'].tolist()
while i<duplicity:
    QA = {}
            
    # parameters
    A = random.choice(A_list)
    param_1 = randint(2,100)
    param_2 = randint(3,1000)
    k = random.choice(measurement_height)
    parameters = [A,str(param_1),str(param_2),k,A,k]
    
    #questions
    QA["question"] = replace_parameters(template_10,parameters)    
 
    # answer
    ans_t10 = randint(param_2+1,100000)

    #options
    options_list = [ans_t10,randint(0,param_2-1),param_2,int(param_2/param_1)]
    random.shuffle(options_list)
    QA["options"] = options_list 
    QA["answer"] = ans_t10
   
    json_dataset.append(QA)
    
    i+=1;

In [961]:
#***********************************************************************************************************#
#  Template 11 : Based on collective nouns                                                                 *#
#***********************************************************************************************************#
data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Collective_noun':str,'Persons':str,'Action':str},sheet_name='template_11')  
df = pd.DataFrame(data,columns=['Collective_noun','Persons','Action'])

#Collective noun consists of atleast two items/people of the same type. So if x number of people/items join, then the total
# number of people could be atleast x+2

template_11 = "In a $A of $B, if $1 $B $C then how many $B could be the present in the $A? "

i=0
A_list =df['Collective_noun'].tolist()
B_list =df['Persons'].tolist()
C_list = df['Action'].tolist()
while i<duplicity:
    QA = {}
            
    # parameters 
    index = randint(0,len(A_list)-1)
    A = A_list[index]  
    B = B_list[index] 
    C = C_list[index] 
    param_1 = randint(2,100000)
    
    parameters = [A,B,str(param_1),B,C,B,A]
    
    #questions
    QA["question"] = replace_parameters(template_11,parameters)    

    # answer
    ans_t11 = randint(param_1+2,100000+param_1)

    #options
    options_list = [ans_t11,param_1,param_1+1,randint(0,param_1)]
    random.shuffle(options_list)
    QA["options"] = options_list 
    QA["answer"] = ans_t11
 
    json_dataset.append(QA)
    
    i+=1;

In [962]:
#***********************************************************************************************************#
#  Template 12 : Based on nouns describing capacity                                                        *#
#***********************************************************************************************************#

data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Capacity':str,'Items':str},sheet_name='template_12')  
df = pd.DataFrame(data,columns=['Capacity','Items']) # Nouns sorted by capacity



template_12 = "If the weight of $A of $C is $1 $k. What could be the weight of $B of $C in $k? "

i=0
while i<duplicity:
    QA = {}
            
    # parameters
    A_list =df['Capacity'].tolist()
    A = random.choice(A_list)
    B_list =[] 
    B_list = A_list.copy()
    B_list.remove(A)
    B = random.choice(B_list)
    k = random.choice(measurement_weight)
    param_1 = randint(2,100000)
    C_list = df['Items'].tolist()
    C = random.choice(C_list)
    
    parameters = [A,C,str(param_1),k,B,C,k]
    
    #questions
    QA["question"] = replace_parameters(template_12,parameters)    
 
    # answer
    index_1 = A_list.index(A)
    index_2 = A_list.index(B)
    if index_1 > index_2: # B is smaller than A
        ans_t12 = randint(1,param_1-1)
        option_2 = randint(param_1+1,100000+param_1)
    else: # B is larger than A
        ans_t12 = randint(param_1+1,100000+param_1)
        option_2 = randint(1,param_1-1)
    

    #options
    options_list = [ans_t12,option_2,param_1]
    random.shuffle(options_list)
    option_4 = random.choice(options_set)
    options_list.append(option_4)
    QA["options"] = options_list 
    QA["answer"] = ans_t12

    json_dataset.append(QA)
    
    i+=1;

In [963]:
#***********************************************************************************************************#
#  Template 13 : With and without                                                                          *#
#***********************************************************************************************************#

data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Items':str,'With':str,'Without':str,"Affects":str},sheet_name='template_13')  
df = pd.DataFrame(data,columns=['Items','With','Without','Affects'])



template_13 = "If the $l of $A $B is $1 $k. How much is the $l of $A $C? "

i=0
A_list =df['Items'].tolist()
X_list = df['With'].tolist()
Y_list = df['Without'].tolist()
l_list = df['Affects'].tolist()
while i<duplicity:
    QA = {}
            
    # parameters
    index = randint(0,len(A_list)-1)
    A = A_list[index]
    l = l_list[index]
    choice = randint(0,1)
    if choice==0:
        B = X_list[index]
        C = Y_list[index]
    else:
        B = Y_list[index]
        C = X_list[index]
    k = random.choice(measurement_weight)
    param_1 = randint(3,100000)
    
    parameters = [l,A,B,str(param_1),k,l,A,C]
    
    #questions
    QA["question"] = replace_parameters(template_13,parameters)    

    # answer
    if choice == 0: #  C < B
        ans_t13 = randint(1, param_1-1)
        option_2 = randint(param_1+1,1000000) 
    else: #weight of C> weight of B
        ans_t13 = randint(param_1+1,1000000)
        option_2 = randint(1, param_1-1)

    #options
    options_list = [ans_t13,option_2,param_1,0]
    random.shuffle(options_list)
    QA["options"] = options_list 
    QA["answer"] = ans_t13
  
    json_dataset.append(QA)
    
    i+=1;


In [964]:
#***********************************************************************************************************#
#  Template 14 : Calculate the earnings                                                                     *#
#***********************************************************************************************************#

data = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Names': str},sheet_name='names')  
df = pd.DataFrame(data,columns=['Names'])   
names = df['Names'].tolist()
template_14 = "$1 has $2 $3. $4 gives the $5 on rent, each for $6 dollars per day. On $7 he does not take any money for them. In a week, how much does $1 earn from $8 $9?"
i=0
while i<duplicity:
    QA = {}
    
    # parameters
    name = random.choice(names)
    no_of_items = randint(2,20)
    item = random.choice(items_for_rent_list)
    price = no_of_items*randint(1,100)
    day = random.choice(days_in_a_week)
    no_of_items_in_qn = randint(1,no_of_items) 
    parameters = [name, str(no_of_items),item,name,item,str(price),day,name,str(no_of_items_in_qn),item]
    
    #questions
    QA["question"] = replace_parameters(template_14,parameters)    

  
    # answers
    earnings = int(price/no_of_items)*6*no_of_items_in_qn
    
    #options
    options_list = [earnings,randint(1,100),randint(0,price),randint(price,earnings*5)]
    random.shuffle(options_list)
    QA["options"] = options_list
    
    QA["answer"] = earnings

    json_dataset.append(QA)
    i+=1;
    

In [965]:
template_15 = "$1 has small $2. $3 uses $4 of them to fill one large $5 of capacity $6 $7. How much is the capacity of one small $8 in $9?"
i=0
while i<duplicity:
    QA = {}
    
    # parameters
    name = random.choice(names)
    container = random.choice(containers_list)
    no_of_small_containers = randint(2,20)
    capacity_of_large_container = no_of_small_containers*randint(1,30)
    measure = random.choice(measurement_liquid)
    parameters = [name, container,name, str(no_of_small_containers), container,str(capacity_of_large_container),measure,container,measure]
  
    #questions
    QA["question"] = replace_parameters(template_15,parameters)    

    # answer
    capacity_of_small_container= int(capacity_of_large_container/no_of_small_containers)
    
    #options
    options_list = [capacity_of_small_container,capacity_of_large_container,randint(0,capacity_of_small_container-1),randint(capacity_of_small_container+1,capacity_of_large_container)]
    random.shuffle(options_list)
    QA["options"] = options_list
    
    QA["answer"] = capacity_of_small_container
    
    json_dataset.append(QA)
    
    i+=1;

In [966]:
template_16 = "$1 bought $2 $3 of $4 from the market for $5 dollars. How much more does $6 need to pay to buy a total of $7 $8 of $9?"
i=0
while i<duplicity:
    QA = {}
    
    # parameters
    name = random.choice(names)
    purchased_items = randint(2,50)
    measure = random.choice(measurement_weight)
    veggie = random.choice(vegetables)
    price = randint(2,50) * purchased_items
    no_of_items = randint(purchased_items+1,100) 
    parameters = [name, str(purchased_items),measure,veggie, str(price),name,str(no_of_items), measure, veggie]
  
    #questions
    QA["question"] = replace_parameters(template_16,parameters)    
 
    # answer
    price_ans = int(((price/purchased_items)*no_of_items) - price)
    
    #options
    options_list = [int(price_ans),price,randint(0,price-1),randint(0,price_ans*6)]
    random.shuffle(options_list)
    QA["options"] = options_list
    
    QA["answer"] = price_ans
    
    json_dataset.append(QA)
    
    i+=1;

In [967]:

template_17 = "If a person who owns a $1 earns $2 dollars in a day, how much will he earn in $3 $4?"
i=0
while i<duplicity:
    QA = {}
    
    # parameters
    owns = random.choice(store_type)
    earns = randint(2,1000)
    duration_time = randint(2,1000) 
    duration = random.choice(duration_list)
    parameters = [owns, str(earns), str(duration_time),duration]
    
    #questions
    QA["question"] = replace_parameters(template_17,parameters)    

    # answer
    if duration == "years":
        t6_ans = earns*30*365*duration_time
    elif duration == "months":
        t6_ans = earns*30*duration_time
    elif duration == "weeks":
        t6_ans = earns*7*duration_time
    else:
        t6_ans = earns*duration_time
        
    
    
    #options
    options_list = [t6_ans,earns,randint(0,earns*5),randint(0,price_ans*6)]
    random.shuffle(options_list)
    QA["options"] = options_list
    
    QA["answer"] = int(t6_ans)
    
    json_dataset.append(QA)
    
    i+=1;


In [968]:
template_18 = "Two $1 plan to $2 along a $3 $4 trail, starting at opposite ends of the trail at the same time. If $P's rate is $5 % $6 than $Q's, how many $7 will $X have walked when they pass each other?"
i=0
while i<duplicity:
    QA = {}
    
    # parameters
    type_of_people = random.choice(human_category)
    activity = random.choice(activity_list)
    length_of_trail = randint(4,1000)
    measure = random.choice(measurement_distance)
    percent = randint(2,50)
    alpha = 'A'
    index = randint(0,24)
    alpha_1 = chr(ord(alpha) + index)
    alpha_2 = chr(ord(alpha) + (index+1))
    alpha_list = [alpha_1,alpha_2]
    question_alpha = random.choice(alpha_list)
    fast_or_slow = ["faster","slower"]
    qn_keyword = random.choice(fast_or_slow)
    
    parameters = [type_of_people, activity, str(length_of_trail),measure,str(alpha_1),str(percent),qn_keyword,str(alpha_2),measure,str(question_alpha)]
    
    #questions
    QA["question"] = replace_parameters(template_18,parameters)    

    # answer
    if qn_keyword == "faster":
        x = (1+(percent/100))*(length_of_trail/(1+(1+(percent/100))))
        if question_alpha == alpha_1:
            t7_ans = x
        else:
            t7_ans = length_of_trail - x
    else:
        x = (1-(percent/100))*(length_of_trail/(1+(1-(percent/100))))
        if question_alpha == alpha_1:
            t7_ans = x
        else:
            t7_ans = length_of_trail - x
    
    
    #options
    options_list = [int(t7_ans),int(length_of_trail/2),randint(0,length_of_trail-3),percent]
    random.shuffle(options_list)
    QA["options"] = options_list
    
    QA["answer"] = int(t7_ans)
    
    json_dataset.append(QA)
    
    i+=1;


In [969]:
Lenght = ["km","m","miles"]
Time = ["sec","hr","min"]
#Since Speed = Length/Time
def convert_length(val,len1,len2):
    if len1 == len2:
        return val
    else:
        if len1=="km" and len2 == "m":
            return val*1000
        elif len1=="m" and len2 == "km":
            return val/1000
        elif len1=="km" and len2 == "miles":
            return val*0.62 
        elif len1=="miles" and len2 == "km":
            return val/0.62
        elif len1== "m" and len2=="miles":
            return val*0.000621371
        else:
            return val/0.000621371
        
def convert_time(val,time1,time2):
    if time1 == time2:
        return val
    else:
        if time1=="min" and time2 == "sec":
            return val/60
        elif time2=="min" and time1 == "sec":
            return val*60
        elif time2=="hr" and time1 == "min":
            return val/60
        elif time1=="hr" and time2 == "min":
            return val*60 
        elif time1=="hr" and time2 == "sec":
            return val/(60*60)
        else:
            return val*(60*60)
                

template_19 = "The speed at which a man can row a boat in still water is $1 $Xp$Y. If he rows $Y, where the speed of current is $2 $Xp$Y, what time in $W will he take to cover $3 $Z?"
i=0
while i<duplicity:
    QA = {}
    
    # parameters
    speed_of_man = randint(4,1000)
    measure_length = random.choice(Lenght)
    measure_time = random.choice(Time)
    directions=["upstream","downstream"]
    dir = random.choice(directions)
    speed_of_curr = randint(2,speed_of_man-1)
    dist = randint(5,100)
    dist_length = random.choice(Lenght)
    dist_time = random.choice(Time)
    
    parameters = [str(speed_of_man), measure_length,measure_time, dir, str(speed_of_curr), measure_length,measure_time,dist_time,str(dist),dist_length]
    
    #questions
    QA["question"] = replace_parameters(template_19,parameters)    

    # answer
    if dir == "downstream":
        x = speed_of_man+speed_of_curr
        L = convert_length(x,measure_length,dist_length)  
        T = convert_time(L,measure_time,dist_time)       
           
    else:
        x = speed_of_man-speed_of_curr
        L = convert_length(x,measure_length, dist_length)  
        T = convert_time(L,measure_time,dist_time)
        
    t8_ans = int(dist/T)   
    #options
    options_list = [int(t8_ans),speed_of_man-speed_of_curr,speed_of_man+speed_of_curr,speed_of_curr]
    random.shuffle(options_list)
    QA["options"] = options_list
    
    QA["answer"] = int(t8_ans)
    
    if t8_ans != 0:
        json_dataset.append(QA)
    
    i+=1;

In [970]:
template_20 = "In the coordinate plane, points ($1, $x) and ($y, $2) are on line $a. If line $a passes through the origin and has slope $3, then what are the values of $x and $y respectively?"
i=0
while i<duplicity:
    QA = {}
    
    # parameters
    alpha = 'a'
    index = randint(0,24)
    alpha_1 = chr(ord(alpha) + index)
    alpha_2 = chr(ord(alpha) + (index+1))
    big_aplha = 'A'
    index_line = randint(0,24)
    aplha_3 = chr(ord(alpha) + index_line)
    x1 = randint(0,10)
    y2 = randint(0,20)
    slope = random.random()
    
    parameters = [str(x1),str(alpha_1),str(alpha_2),str(y2),str(aplha_3),str(aplha_3),str(slope),str(alpha_1),str(alpha_2)]
    
    #questions
    QA["question"] = replace_parameters(template_20,parameters)    

    # answer
    alpha1_x = randint(1,20)
    alpha2_y = int(alpha1_x*slope)
    
    
    #options
    options_list = [str(alpha1_x) +"and"+str(alpha2_y),str(alpha1_x+2) +"and"+str(alpha2_y-1),str(0)+"and"+str(alpha2_y-8),str(int(slope))+"and"+str(0)]
    random.shuffle(options_list)
    QA["options"] = options_list
    
    QA["answer"] = str(alpha1_x)+"and"+str(alpha2_y)
    
    json_dataset.append(QA)
    
    i+=1;

In [971]:
food_expenses = ["food","lunch","dinner","meal","breakfast","coffee","buffet"]
template_21 = "If $1 had $2 at $3 and he gave $4% tip, how much did he spend?"
i=0
while i<duplicity:
    QA = {}
    
    # parameters
    name = random.choice(names)
    expense_item = random.choice(food_expenses)
    price = randint(10,500)
    tip_percent = randint(1,50)
    
    parameters = [name,expense_item,str(price),str(tip_percent)]
    
    #questions
    QA["question"] = replace_parameters(template_21,parameters)    

    # answer
    money_spent = int(price+price*(tip_percent/100))  
    
    
    #options
    options_list = [money_spent, randint(0,money_spent-1),price,price*randint(0,10)]
    random.shuffle(options_list)
    QA["options"] = options_list
    
    QA["answer"] = money_spent
    
    json_dataset.append(QA)
    
    i+=1;

In [972]:
#***********************************************************************************************************#
#  Template 22 : Based on keywords gets, removes, takes away etc                                          *#
#***********************************************************************************************************#
data_1 = pd.read_excel('inputs.xlsx', index_col=None, dtype={'Item': str,'Container':str},sheet_name='template_8')  
df_1 = pd.DataFrame(data_1,columns=['Item','Container'])

# based on keywords get/put
Keywords_22 = ["gets","removes","takes away","puts","places in","adds"]
split_index = 2
template_22 = "In a $A if $X $k $1 $B and $Y $l $2 $B, then how many $B could be present in $A? "

i=0
while i<duplicity:
    QA = {}
            
    # parameters
    A_list =df_1['Container'].tolist()
    index = randint(0,len(A_list)-1)
    A = A_list[index]
    B_list =df_1['Item'].tolist()  
    B = B_list[index]
    X = random.choice(names)
    Y = random.choice(names)
    k = random.choice(Keywords_22)
    l=random.choice(Keywords_22)
    param_1 = randint(2,100000)
    param_2 = randint(2,100000)
    
    parameters = [A,X,k,str(param_1),B,Y,l,str(param_2),B,B,A]
    
    #questions
    QA["question"] = replace_parameters(template_22,parameters)    

    # answer
    index_1 = Keywords_11.index(k)
    index_2 = Keywords_11.index(l)
    if index_1 <= split_index and index_2 <= split_index:
        ans_t22 = randint(param_1+param_2,1000000)
        option_2 = param_1+param_2-1
        option_3 = (param_1+param_2) - randint(2,1000)
    elif index_1 <= split_index and index_2 > split_index:
        ans_t22 = randint(param_2,1000000)
        option_2 = param_2-1
        option_3 = param_2 - randint(0,1000)
    elif index_1 > split_index and index_2 <= split_index:
        ans_t22 = randint(param_1,1000000)
        option_2 = param_1-1
        option_3 = param_1 - randint(0,1000)
    else:
        ans_t22 = randint(param_1+param_2,1000000)
        option_2 = param_1+param_2-1
        option_3 = (param_1+param_2) - randint(2,1000)

    #options
    options_list = [ans_t22,option_2,option_3]
    random.shuffle(options_list)
    option_4 = random.choice(options_set)
    options_list.append(option_4)
    QA["options"] = options_list 
    QA["answer"] = ans_t22
    
    json_dataset.append(QA)
    
    i+=1;

In [973]:
import json
random.shuffle(json_dataset)
with open('dataset.json','w',encoding = 'utf-8') as f:
    json.dump(json_dataset,f,ensure_ascii=False,indent=4)